In [1]:
from pronto import Ontology
import json
import pandas as pd

In [2]:
do = Ontology('https://raw.githubusercontent.com/DiseaseOntology/HumanDiseaseOntology/master/src/ontology/HumanDO.obo')

In [3]:
def read_term (i):
    ii = str(i).replace('Term(','').replace(')','').replace("'",'').split(',')
    doid = ii[0].replace('DOID:','')
    return doid
    

In [4]:
def read_xref (j):
    jj = set(j)
    for n in jj:
        xref = str(n).replace('Xref','').replace('(','').replace(')','').replace("'",'').split(':')
        if 'UMLS_CUI' in xref[0]:
            umls = xref[1]
            return umls

In [5]:
def do_umls(do):
    mapp = {}
    for i in do.terms():
        doid = read_term(i)
        umls = read_xref(i.xrefs)
        if umls is not None:
            mapp[umls] = doid
    return mapp


In [6]:
do2umls = do_umls(do)
with open('do2umls.json','w') as f:
    json.dump(do2umls,f)

In [7]:
repo = pd.read_csv('/home/lxu/Downloads/full.csv')

In [8]:
repo.head()

,drug_name,drug_id,ind_name,ind_id,NCT,status,phase,DetailedStatus
0,Lepirudin,DB00001,Heparin-induced thrombocytopenia with thrombosis,C0272275,NaN,Approved,NaN,NaN
1,Cetuximab,DB00002,Squamous cell carcinoma of mouth,C0585362,NaN,Approved,NaN,NaN
2,Cetuximab,DB00002,Squamous cell carcinoma of nose,C3163899,NaN,Approved,NaN,NaN
3,Cetuximab,DB00002,Squamous cell carcinoma of pharynx,C1319317,NaN,Approved,NaN,NaN
4,Cetuximab,DB00002,Laryngeal Squamous Cell Carcinoma,C0280324,NaN,Approved,NaN,NaN


In [9]:
doid = []
for _,row in repo[['ind_id']].iterrows():
    if row['ind_id'] in do2umls.keys():
        
        do_id = do2umls[row['ind_id']]
        
        doid.append(do_id)
    else:
        doid.append(None)
    


In [10]:
repo.insert(4,'do_id',doid)


In [11]:
nodes = pd.read_csv('/home/lxu/Downloads/integrate-master/data/edges.sif.gz','\t')

In [12]:
nodes.head()

,source,metaedge,target
0,Gene::9021,GpBP,Biological Process::GO:0071357
1,Gene::51676,GpBP,Biological Process::GO:0098780
2,Gene::19,GpBP,Biological Process::GO:0055088
3,Gene::3176,GpBP,Biological Process::GO:0010243
4,Gene::3039,GpBP,Biological Process::GO:0006898


In [14]:
compound = []
disease =[]
for _,row in nodes[['source','target']].iterrows():
    if 'Compound' in row['source']:
        compound.append(row['source'])
    elif 'Disease' in row['source']:
        disease.append(row['source'])
    elif 'Compound' in row['target']:
        compound.append(row['target'])
    elif 'Disease' in row['target']:
        disease.append(row['target'])
    


In [15]:
compound = list(set(compound))

In [16]:
if 'Compound::DB01377' in compound:
    print('1')

In [17]:
disease = list(set(disease))

In [18]:
drug_in_hetio = []
disease_in_hetio = []
for _, row in repo[['drug_id','do_id']].iterrows():
    if 'Compound::'+row['drug_id'] in compound:
        drug_in_hetio.append(1)
    else:
        drug_in_hetio.append(0)
    if row['do_id'] and 'Disease::DOID:'+row['do_id'] in disease:
        disease_in_hetio.append(1)
    else:
        disease_in_hetio.append(0)
    

In [19]:
disease_in_hetio.count(1)

708

In [20]:
repo.insert(2,'drug_in_hetio',drug_in_hetio)
repo.insert(5,'disease_in_hetio',disease_in_hetio)

In [21]:
repo.head()

,drug_name,drug_id,drug_in_hetio,ind_name,ind_id,disease_in_hetio,do_id,NCT,status,phase,DetailedStatus
0,Lepirudin,DB00001,0,Heparin-induced thrombocytopenia with thrombosis,C0272275,0,None,NaN,Approved,NaN,NaN
1,Cetuximab,DB00002,0,Squamous cell carcinoma of mouth,C0585362,0,None,NaN,Approved,NaN,NaN
2,Cetuximab,DB00002,0,Squamous cell carcinoma of nose,C3163899,0,None,NaN,Approved,NaN,NaN
3,Cetuximab,DB00002,0,Squamous cell carcinoma of pharynx,C1319317,0,None,NaN,Approved,NaN,NaN
4,Cetuximab,DB00002,0,Laryngeal Squamous Cell Carcinoma,C0280324,0,2876,NaN,Approved,NaN,NaN


In [22]:
repo_hetio = []
for _,row in repo[['drug_id','drug_in_hetio','disease_in_hetio','do_id','status']].iterrows():
    if row['drug_in_hetio'] ==1 and row['disease_in_hetio'] == 1:
        if row['status'] == 'Approved':
            drug = 'Compound::' + row['drug_id']
            disease = 'Disease::DOID:' + row['do_id']
            repo_hetio.append((drug,disease,1))
        else:
            repo_hetio.append((drug,disease,0))

In [23]:
df = pd.DataFrame(repo_hetio, columns = ['drug','disease','status'])
df
df.to_csv('repo_data.csv')

In [24]:
df_pos = df.loc[df['status']==1]
df_neg = df.loc[df['status']==0]

In [25]:
len(df_pos)

410

In [26]:
len(df_neg)

188

In [27]:
repo_data = pd.read_csv('repo_data.csv')

In [28]:
repo_data

,Unnamed: 0,drug,disease,status
0,0,Compound::DB00091,Disease::DOID:7148,1
1,1,Compound::DB00137,Disease::DOID:10871,1
2,2,Compound::DB00153,Disease::DOID:11476,1
3,3,Compound::DB00169,Disease::DOID:11476,1
4,4,Compound::DB00177,Disease::DOID:10763,1
5,5,Compound::DB00178,Disease::DOID:10763,1
6,6,Compound::DB00180,Disease::DOID:2841,1
7,7,Compound::DB00182,Disease::DOID:9970,1
8,8,Compound::DB00182,Disease::DOID:8986,1
9,9,Compound::DB00184,Disease::DOID:0050742,1


In [49]:
from urllib.request import urlretrieve

In [51]:
urlretrieve('https://github.com/dhimmel/learn/blob/master/prediction/features/features.tsv.bz2?raw=true','/home/lxu/Downloads/bz2try.bz2')

('/home/lxu/Downloads/bz2try.bz2', <http.client.HTTPMessage at 0x7f867f8f31d0>)